In [56]:
import sys

import pandas as pd
import numpy as np

sys.path.append(r'C:\Users\user\Desktop\github\sotrans_analytics_department')
from source.core.de.serving.datamart import Datamart

# ETL

## Extract

In [57]:
datamart: Datamart = Datamart(
    period=8,
    agg_cat_cols=(
        'wh_name',
        'ca_name',
        'ca_type',
        'ca_status',
        'brand_name_1c',
        'sku_art_num'
    ),
    agg_dgt_cols=(
        'cnt',
    ),
    agg_dt_cols=(
        'year',
        'week'
    )
)

2025-05-14 16:51:08.164 | DEBUG    | source.core.de.serving.datamart.datamart:calc_start_date:301 - Конечная дата анализируемого периода не задана, используется значение из Datamart: 2025-05-13
2025-05-14 16:51:08.165 | SUCCESS  | source.core.de.serving.datamart.datamart:calc_start_date:322 - Начальная дата анализируемого периода успешно рассчитана: 2024-10-01


In [58]:
# Конечный остаток;
datamart.IEnd.add_final()

2025-05-14 16:51:08.173 | DEBUG    | source.core.de.serving.datamart.datamart:__get_interface:255 - Создание интерфейса: IEnd
2025-05-14 16:51:08.174 | SUCCESS  | source.core.de.serving.datamart.datamart:__get_interface:257 - Интерфейс IEnd успешно создан
2025-05-14 16:51:08.174 | SUCCESS  | source.core.de.serving.datamart.modules.utils._validate_data:is_cols_available:87 - Кортеж доступных числовых столбцов: ('cnt',)
2025-05-14 16:51:08.175 | SUCCESS  | source.core.de.serving.datamart.modules.sql.sql_end:get_sql_query_final:49 - SQL-запрос успешно сформирован
2025-05-14 16:51:08.334 | SUCCESS  | source.core.de.serving.datamart.datamart:df:155 - Витрина данных обновлена: (73347, 7)


In [59]:
# Перемещения между филиалами;
datamart.ITransfer.add_transfer()

2025-05-14 16:51:08.341 | DEBUG    | source.core.de.serving.datamart.datamart:__get_interface:255 - Создание интерфейса: ITransfer
2025-05-14 16:51:08.341 | SUCCESS  | source.core.de.serving.datamart.datamart:__get_interface:257 - Интерфейс ITransfer успешно создан
2025-05-14 16:51:08.342 | SUCCESS  | source.core.de.serving.datamart.modules.utils._validate_data:is_cols_available:87 - Кортеж доступных числовых столбцов: ('cnt',)
2025-05-14 16:51:08.342 | SUCCESS  | source.core.de.serving.datamart.modules.sql.sql_transfer:get_sql_query_transfer:49 - SQL-запрос успешно сформирован
2025-05-14 16:51:08.557 | SUCCESS  | source.core.de.serving.datamart.datamart:df:155 - Витрина данных обновлена: (74890, 8)


In [60]:
datamart.IArrival.add_arrival()

2025-05-14 16:51:08.562 | DEBUG    | source.core.de.serving.datamart.datamart:__get_interface:255 - Создание интерфейса: IArrival
2025-05-14 16:51:08.563 | SUCCESS  | source.core.de.serving.datamart.datamart:__get_interface:257 - Интерфейс IArrival успешно создан
2025-05-14 16:51:08.563 | SUCCESS  | source.core.de.serving.datamart.modules.utils._validate_data:is_cols_available:87 - Кортеж доступных числовых столбцов: ('cnt',)
2025-05-14 16:51:08.564 | SUCCESS  | source.core.de.serving.datamart.modules.sql.sql_arrival:get_sql_query_arrival:49 - SQL-запрос успешно сформирован
2025-05-14 16:51:08.752 | SUCCESS  | source.core.de.serving.datamart.datamart:df:155 - Витрина данных обновлена: (74906, 9)


In [61]:
datamart.IExpend.add_sale()

2025-05-14 16:51:08.757 | DEBUG    | source.core.de.serving.datamart.datamart:__get_interface:255 - Создание интерфейса: IExpend
2025-05-14 16:51:08.758 | SUCCESS  | source.core.de.serving.datamart.datamart:__get_interface:257 - Интерфейс IExpend успешно создан
2025-05-14 16:51:08.758 | SUCCESS  | source.core.de.serving.datamart.modules.utils._validate_data:is_cols_available:87 - Кортеж доступных числовых столбцов: ('cnt',)
2025-05-14 16:51:08.759 | SUCCESS  | source.core.de.serving.datamart.modules.sql.sql_expend:get_sql_query_sale:254 - SQL-запрос успешно сформирован
2025-05-14 16:51:09.126 | SUCCESS  | source.core.de.serving.datamart.datamart:df:155 - Витрина данных обновлена: (121226, 10)


## Transform

### Pre-Processing

In [62]:
# Замена пропусков в категориальных столбцах;
for col in (
        'wh_name',
        'ca_name',
        'ca_type',
        'ca_status',
        'brand_name_1c',
        'sku_art_num'
):
    datamart.df[col] = datamart.df[col].fillna(value='_нет данных')

In [63]:
# Замена пропусков в количественных столбцах;
for col in (
        'en_final_cnt_fd',
        'tr_transfer_cnt_fd',
        'ar_arrival_cnt_fd',
        'ex_sale_cnt_sum_8_week',
):
    datamart.df[col] = datamart.df[col].fillna(value=0)

In [64]:
# Классификация дилер/перекуп;
condition = (
    datamart.df['ca_type']
    .isin(values=['дилерский контракт', 'плановые закупки в рф'])
)

datamart.df['ca_type'] = (
    np.where(
        condition,
        'дилерский',
        'перекуп'
    )
)

In [65]:
# Предварительная фильтрация;
datamart.df = datamart.df.query(
    expr="ca_type == 'дилерский'"
    " and ca_status == 'работаем'"
)

2025-05-14 16:51:09.365 | SUCCESS  | source.core.de.serving.datamart.datamart:df:155 - Витрина данных обновлена: (41482, 10)


In [66]:
datamart.df = (
    datamart.df
    .groupby(
        by=[
            'wh_name',
            'brand_name_1c',
            'sku_art_num'
        ]
    )
    .aggregate(
        {
            'en_final_cnt_fd': 'sum',
            'tr_transfer_cnt_fd': 'sum',
            'ar_arrival_cnt_fd': 'sum',
            'ex_sale_cnt_sum_8_week': 'sum'
        }
    )
    .reset_index()
)

2025-05-14 16:51:09.415 | SUCCESS  | source.core.de.serving.datamart.datamart:df:155 - Витрина данных обновлена: (38164, 7)


In [67]:
# Расчёт количества недель с наличием;
datamart.IHaving.add_stock(
    agg_cat_cols=(
        'wh_name',
        'brand_name_1c',
        'sku_art_num'
    )
)

2025-05-14 16:51:09.422 | DEBUG    | source.core.de.serving.datamart.datamart:__get_interface:255 - Создание интерфейса: IHaving
2025-05-14 16:51:09.423 | SUCCESS  | source.core.de.serving.datamart.datamart:__get_interface:257 - Интерфейс IHaving успешно создан
2025-05-14 16:51:09.423 | SUCCESS  | source.core.de.serving.datamart.modules.utils._validate_data:is_cols_available:87 - Кортеж доступных числовых столбцов: ('cnt',)
2025-05-14 16:51:09.423 | SUCCESS  | source.core.de.serving.datamart.modules.sql.sql_having:get_sql_query_stock:46 - SQL-запрос успешно сформирован
2025-05-14 16:51:09.571 | SUCCESS  | source.core.de.serving.datamart.datamart:df:155 - Витрина данных обновлена: (103279, 8)


### Feature Engineering

In [68]:
# Расчёт фактического остатка с учётом перемещений и приёмки товаров;
datamart.df['fact_cnt'] = (
    datamart.df['en_final_cnt_fd']
    + datamart.df['tr_transfer_cnt_fd']
    + datamart.df['ar_arrival_cnt_fd']
)

In [69]:
datamart.df['mean_month_sales'] = (
    np.where(
        datamart.df['hv_stock_8_week'] > 0,
        (datamart.df['ex_sale_cnt_sum_8_week'] / datamart.df['hv_stock_8_week']) * 4.5,
        (datamart.df['ex_sale_cnt_sum_8_week'] / 25) * 4
    ) * 1.5
)

In [70]:
# Доступно к перемещению;
condition = datamart.df['fact_cnt'] > datamart.df['mean_month_sales']

datamart.df['move_to_spb'] = np.where(
    condition,
    datamart.df['fact_cnt'] - datamart.df['mean_month_sales'],
    0
)

### Separating

In [71]:
# Вспомогательный DataFrame; только Центральный Склад;
df_wh_spb: pd.DataFrame = datamart.df.query(
    expr="wh_name == '_склад санкт-петербург (красный бор)'")

In [72]:
# Вспомогательный DataFrame; только Центральный Склад;
df_shops: pd.DataFrame = datamart.df.query(
    expr="wh_name != '_склад санкт-петербург (красный бор)' "
    "and wh_name != '_не используется'"
)

### Group by

In [73]:
pt_wh_spb: pd.DataFrame = (
    df_wh_spb
    .groupby(by=['brand_name_1c', 'sku_art_num'])
    .aggregate(
        {
            'en_final_cnt_fd': 'sum',
            'tr_transfer_cnt_fd': 'sum',
            'ar_arrival_cnt_fd': 'sum',
            'ex_sale_cnt_sum_8_week': 'sum',
            'hv_stock_8_week': 'mean',
            'fact_cnt': 'sum',
            'mean_month_sales': 'sum'
        }
    )
    .reset_index()
)

In [74]:
pt_shops: pd.DataFrame = (
    df_shops
    .groupby(by=['wh_name', 'brand_name_1c', 'sku_art_num'])
    .aggregate(
        {
            'ex_sale_cnt_sum_8_week': 'sum',
            'fact_cnt': 'sum',
            'mean_month_sales': 'sum',
            'move_to_spb': 'sum'
        }
    )
    .reset_index()
)

In [75]:
result_table: pd.DataFrame = pd.merge(
    left=pt_wh_spb,
    right=(
        pt_shops
        .rename(
            columns={
                'fact_cnt': 'shop_fact_cnt',
                'mean_month_sales': 'shop_mean_month_sales'
            }
        )
    ),
    how='inner',
    on=['brand_name_1c', 'sku_art_num']
)

In [76]:
result_table

,brand_name_1c,sku_art_num,en_final_cnt_fd,tr_transfer_cnt_fd,ar_arrival_cnt_fd,ex_sale_cnt_sum_8_week_x,hv_stock_8_week,fact_cnt,mean_month_sales,wh_name,ex_sale_cnt_sum_8_week_y,shop_fact_cnt,shop_mean_month_sales,move_to_spb
0,1 new,st55,0.0,0.0,0.0,0.0,30.0,0.0,0.0,магазин великий новгород,0.0,5.0,0.000000,5.000000
1,1 new,st55,0.0,0.0,0.0,0.0,30.0,0.0,0.0,магазин воронеж,0.0,7.0,0.000000,7.000000
2,1 new,st55,0.0,0.0,0.0,0.0,30.0,0.0,0.0,магазин нижний новгород 1,2.0,0.0,0.675000,0.000000
3,1 new,st55,0.0,0.0,0.0,0.0,30.0,0.0,0.0,магазин ростов-на-дону,0.0,6.0,0.000000,6.000000
4,1 new,st55,0.0,0.0,0.0,0.0,30.0,0.0,0.0,магазин санкт-петербург (шушары 2),0.0,2.0,0.000000,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45285,химприма,699509,0.0,0.0,0.0,0.0,1.0,0.0,0.0,магазин уфа,3.0,76.0,1.265625,74.734375
45286,химприма,699509,0.0,0.0,0.0,0.0,1.0,0.0,0.0,магазин челябинск,0.0,9.0,0.000000,9.000000
45287,чмз,1004105ms2912102,0.0,0.0,0.0,0.0,30.0,0.0,0.0,магазин мурманск,0.0,0.0,0.000000,0.000000
45288,чмз,902404mn6505290210110,0.0,0.0,0.0,0.0,30.0,0.0,0.0,магазин мурманск,0.0,0.0,0.000000,0.000000


## Load

In [77]:
result_table.to_json(path_or_buf='move_to_wh_spb.json', orient='table', index=False)